# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core.dataset import Dataset
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Environment
import pandas as pd
import numpy as np


from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException
cluster_name = "azuremlCluster"
try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)


InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
# Utility function to clean data from dataset

def clean_data(data):
    # Clean and one hot encode data
    x_df = data.replace('N/A', np.nan).dropna()
    x_df.drop('id', inplace=True, axis=1)
    x_df['bmi'] = x_df['bmi'].astype(float)

    return (x_df)

In [3]:
dataset_name = 'stroke-dataset'
dataset_description = ''' https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset?select=healthcare-dataset-stroke-data.csv
According to the World Health Organization (WHO) stroke is the 2nd leading cause of death globally, responsible for approximately 11% of total deaths.
This dataset is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status. Each row in the data provides relavant information about the patient.
'''
start_filename = 'healthcare-dataset-stroke-data.csv'
out_filename = 'stroke_data_cleaned.parquet'

if dataset_name in ws.datasets.keys():
    dataset = ws.datasets[dataset_name]
else:
    ds = ws.get_default_datastore()
    df = pd.read_csv(start_filename)
    clean_df = clean_data(df)
    clean_df.to_parquet(out_filename, index=False)
    ds.upload_files(
        files=[
            out_filename
        ],
        target_path='stroke_data',
        overwrite=True,
        show_progress=True
    )

    dataset = Dataset.Tabular.from_parquet_files(path=(ds,'stroke_data/*.parquet'))

    dataset.register(
        workspace=ws,
        name=dataset_name,
        description=dataset_description
    )

# https://towardsdatascience.com/azure-machine-learning-service-where-is-my-data-pjainani-86a77b93ab52

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
experiment_name = 'stroke-prediction-training-hyperdrive'

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.

sklearn_env = Environment.get(workspace=ws, name="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu")
experiment=Experiment(ws, experiment_name)

#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling(
    {'C': uniform(0.001, 10.0),
    'max_iter': choice([500, 1000, 1500, 2000, 2500, 5000])}
)

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='.', script='train.py', 
    compute_target=compute_cluster,
    environment=sklearn_env   
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    primary_metric_name="balanced_accuracy", 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40
)

# https://neptune.ai/blog/balanced-accuracy

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
#TODO: Submit your experiment
experiment = Experiment(ws, "HyperdriveExperiment")
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_98fbe758-1e42-4769-916e-3bfeeb90c86a
Web View: https://ml.azure.com/runs/HD_98fbe758-1e42-4769-916e-3bfeeb90c86a?wsid=/subscriptions/adf28760-4783-41dd-bbd5-bbe3210d5965/resourcegroups/ml-ops/workspaces/telemetry-ws&tid=3c2a020d-f343-4ca1-b6ef-91768380352f

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-06-15T02:47:26.892357][GENERATOR][INFO]Trying to sample '40' jobs from the hyperparameter space<END>\n"

Execution Summary
RunId: HD_98fbe758-1e42-4769-916e-3bfeeb90c86a
Web View: https://ml.azure.com/runs/HD_98fbe758-1e42-4769-916e-3bfeeb90c86a?wsid=/subscriptions/adf28760-4783-41dd-bbd5-bbe3210d5965/resourcegroups/ml-ops/workspaces/telemetry-ws&tid=3c2a020d-f343-4ca1-b6ef-91768380352f



{'runId': 'HD_98fbe758-1e42-4769-916e-3bfeeb90c86a',
 'target': 'azuremlCluster',
 'status': 'Completed',
 'startTimeUtc': '2022-06-15T02:47:26.253346Z',
 'endTimeUtc': '2022-06-15T03:12:35.37778Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"balanced_accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9946bfe7-276e-4b5a-bf13-bbae1b10a004',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.34.0',
  'landmark_points': 'null',
  'space_size': 'infinite_space_size',
  'score': '0.8195493251928021',
  'best_child_run_id': 'HD_98fbe758-1e42-4769-916e-3bfeeb90c86a_14',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_98fbe758-1e42-4769-916e-3bfeeb90c86a_14'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configurati

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Balanced Accuracy:', best_run_metrics['balanced_accuracy'])
print(parameter_values)

Best Run Id:  HD_98fbe758-1e42-4769-916e-3bfeeb90c86a_14

 Accuracy: 0.7253086419753086

 Balanced Accuracy: 0.8195493251928021
['--C', '6.703120249466727', '--max_iter', '500']


In [ ]:
#TODO: Save the best model
if "outputs" not in os.listdir():
    os.mkdir("./outputs")
best_run.download_file('outputs/model.joblib', output_file_path='outputs/best_hyperdrive_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

